# Etivity Assignment 3: Object co-localisation
In this assignment we will use the [Deep Descriptor Transforming (DDT) Technique](https://arxiv.org/pdf/1707.06397.pdf) to perform object co-localisation. The object localisation task involves finding the location of the primary object in an image (same object as image classification result). In the object co-localisation task the goal is to find the objects in images all of the same class. As detailed in the DDT paper, this can be achieved in a unsuperivsed manner from the feature map outputs of a pre-trained network. (In the image: Green bounding box is ground truth, Red boxes are obtained from DDT)
![link text](https://github.com/tonyscan6003/CE6003/blob/master/images/etivity3_assignment_img.JPG?raw=true)

This technique (like Class activiation mapping) demonstrates that spatial information is contained with Deep Neural Networks trained for classification. This information can be used to generate region proposals or for direct object detection. The  unsupervised technique proposed in this paper is useful as it avoids the necessity of having bounding box information to adapt the pre-trained network for localisation.

In this Assignment you will perform object co-localisation using the stanford dogs dataset and the pre-trained VGG-16 network. This Jupyter notebook loads the stanford dogs dataset and also sets up the pre-trained Keras VGG-16 model so that the features of the convolutional layer are output. (Output layers of the model can be changed if required).  

In section 4. of the notebook you will implement the DDT algorithm and demonstrate prediction of bounding boxes. (Ground truth bounding boxes are available for comparison with your prediction).

You may find the following information useful:

*   [Sklearn PCA Functions](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
* [Tensorflow resize](https://www.tensorflow.org/api_docs/python/tf/image/resize) (interpolation function)
*   OpenCV Connected Components (PyImageSearch example [link text](https://www.pyimagesearch.com/2021/02/22/opencv-connected-component-labeling-and-analysis/)) Note that if you want to use OpenCv on the output tensors then it will be necessary to need to convert the tensor to a numpy array uisng `.numpy()` and also ensure that have the numpy array in uint8 format `.astype(np.uint8)`
* If you wish to display any of the images from the dataset, please note that they have been processed prior to input to the network. In order to unprocess the image for display, please use: `helper.unprocess_image(img)`

In order to obtain an exemplary grade you will be required to implement the DDT+ algorithm. This will involve use of additional feature map information to refine the position of the bounding box compared to the basic approach (as shown in the images below). You will implement the DDT+ algorithm on the [Pascale Visual Object Classes Challenge](https://homepages.inf.ed.ac.uk/ckiw/postscript/ijcv_voc09.pdf) "Sheep" class. This is more challenging than the stanford dogs dataset, as objects appear at different scales. Note that the DDT+ approach also causes failures in bounding boxes for many images, you can comment on this in your notebook. 

Image: DDT+ Performance (Cyan: DDT+ bouding boxes, Red DDT: bounding boxes, Green: Ground Truth) 
![link text](https://github.com/tonyscan6003/CE6003/blob/master/images/etivity3b_assignment.JPG?raw=true)

* Note: Completed list of VoC Classes 0-airplane, 1-bicycle, 2-bird, 3-boat, 4-bottle, 5-bus, 6-car, 7-cat, 8-chair, 9-cow, 10-dining table, 11-dog, 12-horse, 13-motorbike, 14-person, 15-potted plant, 16-sheep, 17-sofa, 18-train, 19-TV/monitor.










## 1. HouseKeeping
 Clone Repository & Import Packages 

In [1]:
# Clone repository to gain access to helper.py
!git clone https://github.com/tonyscan6003/CE6003.git

fatal: destination path 'CE6003' already exists and is not an empty directory.


In [ ]:
import tensorflow as tf
import CE6003.python.helper as helper
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow_datasets as tfds

from sklearn.decomposition import PCA
import pandas as pd
import numpy

In [ ]:
# Global Variables
HW_trg = helper.myList[0]      # Target Input Image size
batch_size = helper.myList[1]  # Batch Size
data_set = ["stanford_dogs"]   # Dataset (and add class integer to list for VoC))
#data_set = ["voc",16]    # VoC Dataset with Sheep Class

## 2A. Import Dataset

In [ ]:
def get_data(data_set):
  src_train_dataset,info = tfds.load(data_set[0],split='train',with_info=True)
  src_test_dataset,info  = tfds.load(data_set[0],split='test',with_info=True)
  return src_train_dataset, src_test_dataset,info

  
try:
  src_train_dataset_0, src_test_dataset_0,info_0
except:
  src_train_dataset_0, src_test_dataset_0,info_0 = get_data(data_set)

src_train_dataset, src_test_dataset,info = src_train_dataset_0, src_test_dataset_0,info_0
print(info)

## 2B. Create Train and Test dataset splits 

Note that the images from the stanford dog and many other datasets are not uniform in size. The `gen_datasets` function calls other routines from helper.py that scale the images from the dataset so the longest side fits into the 224 x 224 input window size of VGG16. The aspect ratio of the image is preserved, so the shorter side of the image is padded with zeros. The shorter side is randomly translated providing some data augmentation. (Augmentation is not important for this task). The routines also scale ground truth bounding boxes to match the scale and translation of the image.

In [ ]:
train_dataset, test_dataset =helper.gen_datasets(data_set,src_train_dataset,src_test_dataset)

Display Some Training Images with Ground truth Bounding Box

In [ ]:

helper.display_dataset_img(train_dataset)


## 3A. Import and Setup VGG model
For this assignment we will use the convolutional layers of the VGG-16 module. The sturucture & layer names of the VGG-16 can be viewed on [Netscope](https://ethereon.github.io/netscope/#/preset/vgg-16). The layer_names variable allows selection of the output layer(s) of the model.


In [ ]:

# Load base model
def base_vgg_model(layer_names = ['block5_conv3']):
   IMG_SHAPE = (HW_trg, HW_trg, 3)
   base_vgg_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                             include_top=False,
                                             weights='imagenet')
   
   op_list=[base_vgg_model.get_layer(layer).output for layer in layer_names]
   base_model= tf.keras.Model(inputs=base_vgg_model.input, outputs=op_list)

   return base_model


In [ ]:
# Select Output Layers
layer_names = ['block4_conv3', 'block5_conv3', ]

base_model = base_vgg_model(layer_names=layer_names)
base_model.summary()


## 3B. Extract output features maps from Model.

The function `gen_batch_features` in the code cell below applies `n_img` images from the test or training set to the model. The function returns batch tensors of the image, ground truth boxes and features. The 0 dimension of the tensor corresponds to each image/feature pair.

In [ ]:

def gen_batch_features(base_model, train_dataset,n_img):

    cntr = 0
    for img, boxes, obj_cen, labels in train_dataset.take(n_img):
       img = img.to_tensor(shape=[batch_size, HW_trg, HW_trg, 3])
       A = base_model(img)[0]
       B = base_model(img)[1]

       # Append output features 
       if cntr >0:
          op_features_A = tf.concat([op_features_A,A],axis=0)
          op_features_B = tf.concat([op_features_B,B],axis=0)

          img_batch = tf.concat([img_batch,img],axis=0)
          boxes_batch = tf.concat([boxes_batch,boxes],axis=0)
       else:   
          op_features_A= A
          op_features_B= B

          img_batch = img
          boxes_batch = boxes
       cntr +=1  
     
    return img_batch,boxes_batch,op_features_A, op_features_B   

In [ ]:
n_img=100 # Default value.

img_batch_0,     batch_boxes_0,     op_features_00,      op_features_01     =gen_batch_features(base_model, train_dataset,n_img)
img_batch_test_0,batch_boxes_test_0,op_features_test_00, op_features_test_01=gen_batch_features(base_model, test_dataset,n_img)

### Algorithm 1 Finding the largest connected component
Require: The resized indicator matrix P 1 corresponding to
an image I;
1.   Transform P 1 into a binary map ˆP 1,
where ˆp1(i,j) =
{1 if p1(i,j) > 0
0 otherwise ;
2.   Select one pixel p in ˆP 1 as the starting point;
3.   while True do
4.   Use a flood-fill algorithm to label all the pixels in the
connected component containing p;
5.   if All the pixels are labeled then
6.   Break;
7.   end if
8.   Search for the next unlabeled pixel as p;
9.    end while
10.   Obtain the connectivity of the connected components,
and their corresponding size (pixel numbers);
11.   Select the connected component ˆP 1c with the largest pixel
number;
12.   return The largest connected component ˆP 1c .





### Algorithm 2 Deep Descriptor Transforming (DDT)
**Require:** A set of N images containing the common object,
and a pre-trained CNN model F;


1.    Feed these images with their original resolutions into F;

2.   Collect the corresponding convolutional descriptors X1, . . . , XN from the last convolutional layer of F;

3.   Calculate the mean vector  ̄x of all the descriptors using
Eq. 1;
 
4.   Compute the covariance matrix Cov(x) of these deep
descriptors based on Eq. 2;
5.   Compute the eigenvectors ξ1, . . . , ξd of Cov(x);
6.   Select ξ1 with the largest eigenvalue as the main trans-
forming direction;
7.   **repeat**
    
    7.1.   Calculate the indicator matrix P 1 for image I based
on Eq. 3 and Eq. 4;

    7.2.   Resize P 1 into its image’s resolution by nearest inter-
polation;

    7.3.   Collect the largest connected component ˆP 1c of these
positive regions of the resized P 1 by Algo. 1;

    7.4.   Obtain the minimum rectangle bounding box coveringˆP 1c as the prediction;

8.   **until** All the N images are done;
9.   return The minimum rectangle bounding boxes.

## 4. DDT Algorithm

In the code cells below add your implementation of the DDT Algorithm for object co-localisation.


In [ ]:
def describe_features(op_features, verbose=True):
  fs = np.shape(op_features)
  if verbose:
    print(f"{fs[0]} features of {str(fs[1])}{str(''.join(map(str, ['x'+str(i) for i in fs[2:]])))} shape")
  return fs

def reshape_features(op_features, verbose=True):
  fs = describe_features(op_features, verbose=verbose)
  fs_0 = numpy.prod(fs[:len(fs)-1])
  if verbose:
    print(f"reshaping to {fs_0} ({str(''.join(map(str, [str(i)+'x' for i in fs[:len(fs)-2]])))}{str(fs[1])}) samples of {fs[-1]}")
  return tf.reshape(op_features, [fs_0, fs[-1]])

In [ ]:
def print_shapes(lists):
  print("Image Shape Transformations:")
  for l in lists:
    if isinstance(l, str):
      msg=''
      if l[0]=='7':
        msg = "\nstep: "
      print(msg+l)
    elif isinstance(l,(list,pd.core.series.Series,np.ndarray)) or tf.is_tensor(l):
      print(np.shape(l))

def find_largest_connected_component(image_nearest_interpolation):
  #A1
  # 1 -> 9
  binary_img = cv2.threshold(np.uint8(image_nearest_interpolation), 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

  # 10  - Obtain the connectivity of the connected components, and their corresponding size (pixel numbers);
  nb_components, output, stats, centroids  = cv2.connectedComponentsWithStats(binary_img, cv2.CV_32S, connectivity=8)
  # print((np.shape(numLabels), np.shape(labels), np.shape(stats), np.shape(centroids)))

  # 11  - The largest connected component ˆP1c .
  largest_connected_component_0 = stats[np.argsort(stats[:, cv2.CC_STAT_AREA])][0]
  largest_connected_component_1 = stats[np.argsort(stats[:, cv2.CC_STAT_AREA])][1]

  #A1 end
  return nb_components, output, stats,  largest_connected_component_0, largest_connected_component_1, binary_img



def define_bounding_box(nb_components, largest_connected_component):
  if nb_components>0:
    stat_left   = largest_connected_component[cv2.CC_STAT_LEFT]/255 
    stat_top    = largest_connected_component[cv2.CC_STAT_TOP]/255 
    stat_right  = largest_connected_component[cv2.CC_STAT_WIDTH]/255 + stat_left
    stat_bottom = largest_connected_component[cv2.CC_STAT_HEIGHT]/255 + stat_top
    return tf.constant([[stat_left , stat_top, stat_bottom, stat_right]])
  else:
    return tf.constant([[0,0,0,0]])



def get_indicator_matrix(pca, op_feat, feature_transformation_shape_logs):
    op_feats_reshaped = reshape_features(op_feat, verbose=False)
    feature_transformation_shape_logs.extend(["7.1", "\nop_feats_reshaped", op_feats_reshaped])

    pflat = pca.transform(op_feats_reshaped)
    feature_transformation_shape_logs.extend(["\npflat", pflat])

    indicator_matrix_14x14_0 = tf.reshape(pflat, [np.shape(op_feat)[0], np.shape(op_feat)[1]])
    feature_transformation_shape_logs.extend(["\nindicator_matrix_14x14_0", indicator_matrix_14x14_0])

    #  just to keep images being 3D tensors (https://stackoverflow.com/a/58997339/12279207)
    indicator_matrix_14x14_as3D_tensor = indicator_matrix_14x14_0[..., tf.newaxis]
    feature_transformation_shape_logs.extend(["\nindicator_matrix_14x14_1", indicator_matrix_14x14_as3D_tensor])


    return indicator_matrix_14x14_as3D_tensor, feature_transformation_shape_logs
  

def run_pca(pca, op_feats):
  bounding_boxes_0=[]
  bounding_boxes_1=[]

  binary_imgs = []
  indicator_matrices = []
  feature_transformation_shape_logs = [op_feats]

  first=True
  for i in range(len(img_batch_test)):
  #A2
    #7.1 - Calculate the indicator matrix P 1 for image
    indicator_matrix_14x14_1, feature_transformation_shape_logs = get_indicator_matrix(pca, op_feats[i], feature_transformation_shape_logs)

    # 7.2  - Resize P 1 into its image’s resolution by nearest interpolation;
    image_nearest_interpolation = tf.image.resize(indicator_matrix_14x14_1, [224, 224], preserve_aspect_ratio=True, antialias=False, name=None
                                                  ).numpy().astype(np.uint8)

    feature_transformation_shape_logs.extend(["7.2", "\nimage_nearest_interpolation", image_nearest_interpolation])
    indicator_matrices.append(tf.reshape(image_nearest_interpolation, [224,224], name=None))


    image_nearest_interpolation = cv2.normalize(src=image_nearest_interpolation,dst=None,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U)
    image_nearest_interpolation = tf.clip_by_value(image_nearest_interpolation, clip_value_min=0, clip_value_max=255)


    #7.3  - Collect the largest connected component ˆP1c of these positive regions of the resized P1 by Algorithm 1
    (nb_components, output, stats, 
     largest_connected_component_0, largest_connected_component_1, binary_img) = find_largest_connected_component(image_nearest_interpolation)
    binary_imgs.append(binary_img)
    # 7.4   - Obtain the minimum rectangle bounding box
    bounding_boxes_0.append(
        define_bounding_box(nb_components, largest_connected_component_0))
    bounding_boxes_1.append(
        define_bounding_box(nb_components, largest_connected_component_1))

    # 9: return The minimum rectangle bounding boxes    
    if first:
      print_shapes(feature_transformation_shape_logs)
      first=False
  return [bounding_boxes_0, bounding_boxes_1], binary_imgs, indicator_matrices

def Object_coLocalisation(op_features, op_features_test):
  pca_dogs_model = PCA(n_components=1)
  op_features_reshaped = reshape_features(op_features)
  pca_dogs = pca_dogs_model.fit_transform(op_features_reshaped)
  bounding_boxes_DDT, binary_imgs_DDT, indicator_matrices_DDT = run_pca(pca_dogs_model, op_features_test)
  return bounding_boxes_DDT, binary_imgs_DDT, indicator_matrices_DDT



In [ ]:
(bounding_boxes_DDT_0,
 binary_imgs_DDT_0,
 indicator_matrices_DDT_0) = Object_coLocalisation(op_features_00, op_features_test_00)

(bounding_boxes_DDT_1,
 binary_imgs_DDT_1,
 indicator_matrices_DDT_1) = Object_coLocalisation(op_features_01, op_features_test_01)


bounding_boxes_DDT_0_01 = [bounding_boxes_DDT_0, bounding_boxes_DDT_1]
print(np.shape(bounding_boxes_DDT))

## 5. Display results

You can display your results using the code cell below. You can display an image(s) similar to that shown at the start of the notebook, containing the image, ground truth bounding box and the bounding box produced by the DDT method. You can also show the binary map produced by indicator P (positive values).

The function `helper.image_with_gt_boxes(img,boxes,colour)` can be used to plot a bounding box on an image where the bounding box is in the form [ymin,xmin,ymax,xmax] and values are normalised between zero and 1. 

In [ ]:
def draw_bounds(ax, curr_img, title):
  # Plot image and show related indicator
  ax.axis('off')
  ax.set_title(title)
  ax.imshow(curr_img)

def display_img_results():
  ground_truth_img=helper.image_with_gt_boxes(curr_img,batch_boxes_test[k],(0,255,0))
  ax = axs[0][k]
  draw_bounds(ax, ground_truth_img, "Ground Truth")


def display_image_bounding_box_results(k, axs, img_batch_test, batch_boxes_test, bounding_boxes_DDT):
  curr_img = np.asarray(helper.unprocess_image(img_batch_test[k, :, :,:]))
  curr_img=helper.image_with_gt_boxes(curr_img,batch_boxes_test[k],(0,255,0))
  draw_bounds(axs[0][k], curr_img, "Ground Truth")

  for bbs in bounding_boxes_DDT:

    for color, box_type in zip([(255,0,0),(245,218,223)], bbs):
      curr_img=helper.image_with_gt_boxes(curr_img, box_type[k], color)
      draw_bounds(axs[1][k], curr_img, "Prediction")

def display_All_coLocalisation_results(img_batch_test, batch_boxes_test, bounding_boxes_DDT, indicator_matrices_DDT, binary_imgs_DDT, n=8):
  n=8
  fig, axs = plt.subplots(4,n, figsize=(30, 15))

  # Iterate through some images
  for k in range(n):
    display_image_bounding_box_results(k, axs[:2], img_batch_test, batch_boxes_test, bounding_boxes_DDT)
    axs[2][k].imshow(indicator_matrices_DDT[k], cmap='gray')
    axs[2][k].set_title("indicator_matrix")
    axs[2][k].axis('off')
    axs[3][k].imshow(binary_imgs_DDT[k], cmap='gray')
    axs[3][k].set_title("binary_img")
    axs[3][k].axis('off')

display_All_coLocalisation_results(img_batch_test_0, batch_boxes_test_0, bounding_boxes_DDT_0_01, indicator_matrices_DDT_0, binary_imgs_DDT_0, n=8)

## DDT+ Algorithm

In [ ]:
data_set = ['voc', 16]
try:
  src_train_dataset_1, src_test_dataset_1,info_1
except:
  src_train_dataset_1, src_test_dataset_1,info_1 = get_data(data_set)

src_train_dataset, src_test_dataset,info = src_train_dataset_1, src_test_dataset_1,info_1

train_dataset, test_dataset =helper.gen_datasets(data_set,src_train_dataset,src_test_dataset)

helper.display_dataset_img(train_dataset)




In [ ]:
n_img=100 # Default value.



img_batch_1,     batch_boxes_1,     op_features_10,      op_features_11     =gen_batch_features(base_model, train_dataset,n_img)
img_batch_test_1,batch_boxes_test_1,op_features_test_10, op_features_test_11=gen_batch_features(base_model, test_dataset,n_img)


In [ ]:
(bounding_boxes_DDT_0,
 binary_imgs_DDT_0,
 indicator_matrices_DDT_0) = Object_coLocalisation(op_features_10, op_features_test_10)

(bounding_boxes_DDT_1,
 binary_imgs_DDT_1,
 indicator_matrices_DDT_1) = Object_coLocalisation(op_features_11, op_features_test_11)


bounding_boxes_DDT_1_01 = [bounding_boxes_DDT_0, bounding_boxes_DDT_1]
print(np.shape(bounding_boxes_DDT))

In [ ]:
# fig = plt.figure(figsize=(15, 12))
display_All_coLocalisation_results(img_batch_test_1, batch_boxes_test_1, bounding_boxes_DDT_1_01, indicator_matrices_DDT_0, binary_imgs_DDT_0, n=8)